# Large Rocks Detection Project

Welcome to the **Large Rocks Detection Project**! This notebook serves to implement our machine learning pipeline for detecting large rocks. Below is an outline of the steps we will follow throughout the project:

## Outline

1. **Dataset Preparation**  
   Organize and adapt the training, validation, and test datasets using `dataset.py`.
2. **Data Augmentation**  
   Apply geometric and visual transformations for enhanced generalization, leveraging `dataset.py`.
3. **Model Training**  
   Train the model using `model.py`.
4. **Regularization to Combat Overfitting**  
   Employ validation strategies to minimize overfitting.
5. **Evaluation on Test Data**  
   Test the model on the final dataset and visualize the results.
6. **Accuracy Metrics**  
   Calculate and report accuracy metrics for a comprehensive performance evaluation.

Let’s dive into each step and build a robust solution for detecting large rocks!

In [1]:
from tifffile import tifffile 
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image

import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
import torchvision.transforms as T
import os
import dataset as dt

## 1. Dataset Preparation

Download the given data ... (in a folder named 'Data')

In [2]:
# Load dataset from JSON
json_file_path = 'Data/large_rock_dataset.json'
data, dataset = dt.load_dataset_from_json(json_file_path)

# Define which type of images you want to use (here we use RGB and hillshade)
img_folders = ['Data/swissImage_50cm_patches/', 'Data/swissSURFACE3D_hillshade_patches/']

for i in img_folders:
    # Extract folder name from the path
    folder_name = os.path.basename(os.path.normpath(i))

    # Define base directory name where the images and labels will be stored
    base_dir_name = f'dataset_{folder_name}'

    # Split and organize the dataset
    train_images, test = dt.split_train_from_json(dataset, i, base_dir_name)
    train_labels = dt.save_train_annotations(dataset, base_dir_name)

    # Create a validation set
    val_images = dt.create_validation_set_images(train_images, base_dir_name)
    val_labels = dt.create_validation_set_labels(train_labels, base_dir_name)

    # Convert. tif to .jpg for Yolov8
    dt.convert_tif_to_jpg(train_images)
    dt.convert_tif_to_jpg(val_images)

    # Write the labels in Yolov8 format
    # YOLOv8 assumes constant bbox size
    bbox_width = 10 / 640  # Normalized width
    bbox_height = 10 / 640  # Normalized height

    dt.convert_labels_to_yolo_format(
        train_labels,
        base_dir_name,
        bbox_width=bbox_width,
        bbox_height=bbox_height,
        type='train'
    )
    
    dt.convert_labels_to_yolo_format(
        val_labels,
        base_dir_name,
        bbox_width=bbox_width,
        bbox_height=bbox_height,
        type='val'
    )

General information about the data: {'description': 'Large Rocks Detection Dataset ', 'version': '1.0', 'year': 2024, 'contributor': 'Valerie Zermatten', 'date_created': '2024/09/30'}
Copied '2581_1126_0_0.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_0_1.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_0_2.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_0_3.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_1_0.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_1_1.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_1_2.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_1_3.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_2_0.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_2_1.tif' to 'dataset_swissImage_50cm_patches\train_images'
Copied '2581_1126_2_2.tif' to 'dataset_swissIm

## 2. Data Augmentation

### For RGB images

In [5]:
# Initialize and inspect the dataset

# Set paths for training images and YOLO-format labels.
image_folder = "dataset_swissImage_50cm_patches/train_images"
label_folder = "dataset_swissImage_50cm_patches/yolo_train_labels"

# Calculate mean and standard deviation for normalization.
mean, std = dt.calculate_mean_std(image_folder)

# Create the RockDetectionDataset with normalization (no augmentation for now).
dataset = dt.RockDetectionDataset(image_folder, label_folder, mean, std, augment=False)

# Iterate through the dataset to:
#  - Print the image name, tensor shape, and associated labels.
#  - Break after the first iteration for quick inspection.
for idx, (aug_images, aug_labels) in enumerate(dataset):
    image_name = dataset.image_files[idx]  # Get the name of the current image
    print(f"Image Name: {image_name}")
    print(f"Image Shape: {aug_images.size()}")
    print(f"Labels: {aug_labels}")
    break

AssertionError: 

In [ ]:
# Data Augmentation Workflow: Geometric, Brightness, and Obstruction
# This script performs three types of augmentations (Geometric, Brightness, and Obstruction)
# and saves the augmented datasets into separate folders.

# Parameters
batch_size = 16 
base_dir = 'dataset_swissImage_50cm_patches'
augmentations = [
    {"type": "geometric", "function": dt.geometric_augmentations, "output_suffix": "geom"},
    {"type": "brightness", "function": dt.brightning, "output_suffix": "bright"},
    {"type": "obstruction", "function": dt.obstruction, "output_suffix": "obst"}
]

for augmentation in augmentations:
    # Define output paths for the current augmentation
    output_image_folder = os.path.join(base_dir, f"augmented_train_images_{augmentation['output_suffix']}")
    output_label_folder = os.path.join(base_dir, f"augmented_train_labels_{augmentation['output_suffix']}")

    # Batch processing loop
    batch_images, batch_labels, batch_names = [], [], []

    for idx in range(len(dataset)):
        # Load original image and labels
        original_image, original_labels = dataset[idx]
        original_image_name = dataset.image_files[idx]  # Get image name

        # Convert tensor to PIL image if needed
        if isinstance(original_image, torch.Tensor):
            original_image = dt.denormalize(original_image.clone(), mean, std)
            original_image = T.ToPILImage()(original_image)

        # Perform augmentations using the specified function
        augmented_data = augmentation["function"](original_image, original_labels)

        # Append each augmented image and labels to the batch
        for aug_idx, (aug_image, aug_labels) in enumerate(augmented_data):
            batch_images.append([aug_image])  # Wrap in a list for batch saving
            batch_labels.append([aug_labels])
            batch_names.append(f"{original_image_name}_aug_{aug_idx + 1}")

        # When batch is full, save the batch
        if len(batch_images) >= batch_size:
            dt.save_augmented_data_batch(
                batch_images, batch_labels, batch_names,
                output_image_folder, output_label_folder,
                augmentation_names=[f"aug_{i + 1}" for i in range(len(batch_images))],
                mean=mean, std=std
            )
            # Reset batch
            batch_images, batch_labels, batch_names = [], [], []

    # Save remaining images in the last batch
    if len(batch_images) > 0:
        dt.save_augmented_data_batch(
            batch_images, batch_labels, batch_names,
            output_image_folder, output_label_folder,
            augmentation_names=[f"aug_{i + 1}" for i in range(len(batch_images))],
            mean=mean, std=std
        )

print("Data augmentation complete for all types.")

### Fore hillshade images

In [ ]:
# Initialize and inspect the dataset

# Set paths for training images and YOLO-format labels.
image_folder = "dataset_swissSURFACE3D_hillshade_patches/train_images"
label_folder = "dataset_swissSURFACE3D_hillshade_patches/yolo_train_labels"

# Calculate mean and standard deviation for normalization.
mean, std = dt.calculate_mean_std(image_folder)

# Create the RockDetectionDataset with normalization (no augmentation for now).
dataset = dt.RockDetectionDataset(image_folder, label_folder, mean, std, augment=False)

# Iterate through the dataset to:
#  - Print the image name, tensor shape, and associated labels.
#  - Break after the first iteration for quick inspection.
for idx, (aug_images, aug_labels) in enumerate(dataset):
    image_name = dataset.image_files[idx]  # Get the name of the current image
    print(f"Image Name: {image_name}")
    print(f"Image Shape: {aug_images.size()}")
    print(f"Labels: {aug_labels}")
    break

In [ ]:
# Data Augmentation Workflow: Geometric, Brightness, and Obstruction
# This script performs three types of augmentations (Geometric, Brightness, and Obstruction)
# and saves the augmented datasets into separate folders.

# Parameters
batch_size = 16 
base_dir = 'dataset_swissSURFACE3D_hillshade_patches'
augmentations = [
    {"type": "geometric", "function": dt.geometric_augmentations, "output_suffix": "geom"},
    {"type": "brightness", "function": dt.brightning, "output_suffix": "bright"},
    {"type": "obstruction", "function": dt.obstruction, "output_suffix": "obst"}
]

for augmentation in augmentations:
    # Define output paths for the current augmentation
    output_image_folder = os.path.join(base_dir, f"augmented_train_images_{augmentation['output_suffix']}")
    output_label_folder = os.path.join(base_dir, f"augmented_train_labels_{augmentation['output_suffix']}")

    # Batch processing loop
    batch_images, batch_labels, batch_names = [], [], []

    for idx in range(len(dataset)):
        # Load original image and labels
        original_image, original_labels = dataset[idx]
        original_image_name = dataset.image_files[idx]  # Get image name

        # Convert tensor to PIL image if needed
        if isinstance(original_image, torch.Tensor):
            original_image = dt.denormalize(original_image.clone(), mean, std)
            original_image = T.ToPILImage()(original_image)

        # Perform augmentations using the specified function
        augmented_data = augmentation["function"](original_image, original_labels)

        # Append each augmented image and labels to the batch
        for aug_idx, (aug_image, aug_labels) in enumerate(augmented_data):
            batch_images.append([aug_image])  # Wrap in a list for batch saving
            batch_labels.append([aug_labels])
            batch_names.append(f"{original_image_name}_aug_{aug_idx + 1}")

        # When batch is full, save the batch
        if len(batch_images) >= batch_size:
            dt.save_augmented_data_batch(
                batch_images, batch_labels, batch_names,
                output_image_folder, output_label_folder,
                augmentation_names=[f"aug_{i + 1}" for i in range(len(batch_images))],
                mean=mean, std=std
            )
            # Reset batch
            batch_images, batch_labels, batch_names = [], [], []

    # Save remaining images in the last batch
    if len(batch_images) > 0:
        dt.save_augmented_data_batch(
            batch_images, batch_labels, batch_names,
            output_image_folder, output_label_folder,
            augmentation_names=[f"aug_{i + 1}" for i in range(len(batch_images))],
            mean=mean, std=std
        )

print("Data augmentation complete for all types.")

## 3. Model Trainning